European Central Bank (ECB) API with Python
=====

## Data on Eurozone countries

-----

*December 14, 2019*<br>
*@bd_econ*

ECB API Documentation is [here](https://sdw-wsrest.ecb.europa.eu/help/)

This example requests the interest rate by country on 10-year bonds over the period since 2012.

You can read the output from the ECB API (as json data, so with some patience) in a web browser, to better understand what my code below is doing.

In [1]:
import requests
import pandas as pd

## Parameters/Settings

List of data sets is [here](https://sdw-wsrest.ecb.europa.eu/service/dataflow).

You can also find the `series_key` from searching the [Statistics Data Warehouse](https://sdw.ecb.europa.eu/home.do).

In [2]:
# List of countries
c_list = ['DE', 
          'FR', 
          'IT', 
          'ES', 
          'PT', 
          'GR', 
          'NL', 
          'BE', 
          'AT', 
          'IE']

param = [('dataflow', 'IRS'),
         ('freq', 'M'),
         ('countries', '+'.join(c_list)),
         ('series', 'L'),
         ('trans type', 'L40'),
         ('maturity cat', 'CI'),
         ('counterpart', '0000'),
         ('currency', 'EUR'),
         ('business coverage', 'N'),
         ('interest rate type', 'Z'),
         ('start', '?startPeriod=2012-01-01')]

param_joined = '.'.join(value for key, value in param[1:-1])

series_key = '{}/{}{}'.format(param[0][1],
                              param_joined,
                              param[-1][1])

In [3]:
print(series_key)

IRS/M.DE+FR+IT+ES+PT+GR+NL+BE+AT+IE.L.L40.CI.0000.EUR.N.Z?startPeriod=2012-01-01


## Request data

In [4]:
url = 'https://sdw-wsrest.ecb.europa.eu/service/data/'
# headers used as content negotiation to return data in json format
headers = {'Accept':'application/json'}
r = requests.get('{}{}'.format(url, series_key), headers=headers).json()

## Pandas DataFrame

In [5]:
date_list = r['structure']['dimensions']['observation'][0]['values']
dates = [date['start'][:10] for date in date_list]
    
areas = [v['name'] for v in r['structure']['dimensions']['series'][1]['values']]

df = pd.DataFrame()
for i, area in enumerate(areas):
    s_key = '0:{}:0:0:0:0:0:0:0'.format(i)
    s_list = r['dataSets'][0]['series'][s_key]['observations']
    df[area] = pd.Series([s_list[val][0] for val in sorted(s_list, key=int)])
df.index = dates
df.tail()

,Austria,Belgium,Germany,Spain,France,Greece,Ireland,Italy,Netherlands,Portugal
2019-07-01,-0.1037,0.00,-0.39,0.354,-0.07,2.16,0.13,1.654,-0.206,0.443
2019-08-01,-0.3737,-0.28,-0.65,0.141,-0.34,1.98,-0.05,1.404,-0.499,0.173
2019-09-01,-0.3024,-0.24,-0.59,0.180,-0.28,1.50,-0.01,0.902,-0.427,0.204
2019-10-01,-0.1974,-0.16,-0.47,0.203,-0.16,1.34,0.02,1.004,-0.314,0.187
2019-11-01,-0.0861,-0.04,-0.35,0.386,-0.02,1.36,0.07,1.266,-0.192,0.349


### Example 2: Population

In [6]:
series_key2 = 'ENA/A.N.DE+FR+IT+ES+PT+GR+NL+BE+AT+IE.W0.S1.S1._Z.POP._Z._Z._Z.PS._Z.N'

In [7]:
url = 'https://sdw-wsrest.ecb.europa.eu/service/data/'
# headers used as content negotiation to return data in json format
headers = {'Accept':'application/json'}
r = requests.get('{}{}'.format(url, series_key2), headers=headers).json()

In [8]:
date_list = r['structure']['dimensions']['observation'][0]['values']
dates = {i: v['id'] for i, v in enumerate(date_list)}    
areas = [v['name'] for v in r['structure']['dimensions']['series'][2]['values']]

df = pd.DataFrame()
for i, area in enumerate(areas):
    s_key = f'0:0:{i}:0:0:0:0:0:0:0:0:0:0:0'
    s_list = r['dataSets'][0]['series'][s_key]['observations']
    df[area] = pd.Series({dates[int(i)]: v[0] for i, v in s_list.items()})
df.tail()

,Austria,Belgium,Germany,Spain,France,Greece,Ireland,Italy,Netherlands,Portugal
2014,8543.932,11180,80983,46455.123,66312,10892.369,4652.459,60789.1,16865,10401.1
2015,8629.519,11238,81687,46410.149,66581,10820.964,4695.769,60730.6,16940,10358.1
2016,8739.806,11295,82349,46449.874,66828,10775.989,4748.994,60627.5,17030,10325.5
2017,8795.073,11349,82657,46532.869,67063,10754.701,4802.274,60536.7,17131,10300.3
2018,8837.707,11404,82906,46728.962,67274,10727.560,4860.650,60421.8,17232,10283.8
